In [ ]:
! pip install Flask


In [ ]:
! pip install import_ipynb

In [ ]:
! pip install pymysql

In [ ]:
! pip install WTForms

In [ ]:
! pip install mysqlclient

In [ ]:
! pip install flask-mysqldb


In [ ]:
! pip install flask-login

In [ ]:
! pip freeze

In [ ]:
! pip install flask-login

In [ ]:
! pip install Flask-Bcrypt 

In [205]:
from flask import Flask, request, render_template, redirect, url_for, jsonify, flash
import import_ipynb
import forms, hashlib 
from flask_mysqldb import MySQL
from werkzeug.security import generate_password_hash, check_password_hash
from flask_bcrypt import Bcrypt 
from IPython.display import display, HTML
from flask_login import LoginManager,UserMixin,login_user,logout_user,login_required,current_user

In [206]:
# intilize flask application
app = Flask(__name__)
bcrypt = Bcrypt(app) 

In [207]:
app.config["SECRET_KEY"]="Hola como etas muy bein y tu?"
app.config['MYSQL_HOST']="localhost"
app.config["MYSQL_USER"]="root"
app.config["MYSQL_PASSWORD"]=""
app.config["MYSQL_DB"]="schoolapp"

In [208]:
mysql = MySQL(app)

In [209]:
login_manager=LoginManager()
login_manager.init_app(app)
login_manager.login_view="login"

In [210]:
class User(UserMixin) :
    def __init__(self,id,first_name,last_name,middle_name,email,password,gender,mobile_number,whatsapp_number,address,city,state,country,profile_picture,role,status) :
        self.id=id
        self.first_name=first_name
        self.last_name=last_name
        self.middle_name=middle_name
        self.email=email
        self.password=password
        self.gender=gender
        self.mobile_number=mobile_number
        self.whatsapp_number=whatsapp_number
        self.address=address
        self.city=city
        self.state=state
        self.country=country
        self.profile_picture=profile_picture
        self.role=role
        self.status=status


In [211]:
@login_manager.user_loader
def load_user(user_id) :
     sql="select * from users where users_id=%s AND status=%s"
     cursor=mysql.connection.cursor()
     cursor.execute(sql,(user_id, 'active'))
     user = cursor.fetchone()
     cursor.close()
     if user :
          return User(user[0],user[1],user[2],user[3],user[4],user[5],user[6],user[7],user[8],user[9],user[10],user[11],user[12],user[13],user[14],user[15])


In [212]:
@app.route("/")
def home():
    return redirect(url_for("login"))

In [213]:
@app.route("/login",methods=["GET","POST"])
def login():
    if request.method=="POST":
        login_form=forms.LoginForm(request.form)
        if login_form.validate():
            user_data=request.form
            email=user_data["email"]
            password=user_data["password"]
            sql="select * from users where email=%s AND status = %s"
            cursor=mysql.connection.cursor()
            try:
                cursor.execute(sql,(email, 'active'))
                account = cursor.fetchone()
                if account and bcrypt.check_password_hash(account[5], password):
                    user_object=User(account[0],account[1],account[2],account[3],account[4],account[5],account[6],account[7],account[8],account[9],account[10],account[11],account[12],account[13],account[14],account[15])
                    login_user(user_object)
                    flash("Login successful","success")
                    return redirect(url_for("dashboard"))
                else :
                    flash("Invalid login credentials.","error")
                    return redirect(request.referrer)
                cursor.close()
                return jsonify(login_form.data)
            except Exception as e:
                print("Failed to load user", e)
                return "Failed to load user"
            

        else :
            for field,errors in login_form.errors.items():
                for error in errors:
                    flash(f"{field}: {error}","error")
            return redirect(request.referrer)
    else :
        return render_template("auth/user.html") 

In [214]:
@app.route("/dashboard", methods=["GET","POST"])
@login_required
def dashboard():
    print(current_user.role)
    return render_template("pages/dashboard.html", first_name=current_user.first_name, last_name=current_user.last_name, middle_name=current_user.middle_name, email=current_user.email, gender=current_user.gender, mobile_number=current_user.mobile_number, whatsapp_number=current_user.whatsapp_number, address=current_user.address, city=current_user.city, state=current_user.state, country=current_user.country, profile_picture=current_user.profile_picture, role=current_user.role, status=current_user.status )


In [215]:
@app.route("/Courses", methods=["GET", "POST"])
@login_required
def Courses():
    return render_template("pages/Courses.html", first_name=current_user.first_name, last_name=current_user.last_name, middle_name=current_user.middle_name, email=current_user.email, gender=current_user.gender,mobile_number=current_user.mobile_number,whatsapp_number=current_user.whatsapp_number,address=current_user.address,city=current_user.city,state=current_user.state,country=current_user.country,profile_picture=current_user.profile_picture,role=current_user.role,status=current_user.status )

In [216]:
@app.route("/logout", methods=["GET","POST"])
@login_required
def logout():
    logout_user()
    flash("Logged out succesful","Info")
    return redirect(url_for("login"))

In [217]:
@app.route("/Quizzes", methods=["Get","Post"])
@login_required
def Quizzes():
    return render_template("pages/quizzess.html", first_name=current_user.first_name, last_name=current_user.last_name, middle_name=current_user.middle_name, email=current_user.email, gender=current_user.gender,mobile_number=current_user.mobile_number,whatsapp_number=current_user.whatsapp_number,address=current_user.address,city=current_user.city,state=current_user.state,country=current_user.country,profile_picture=current_user.profile_picture,role=current_user.role,status=current_user.status)
 

In [218]:
@app.route("/certificate", methods=["Get","Post"])
@login_required
def Certificate():
    return render_template("pages/certificate.html", first_name=current_user.first_name, last_name=current_user.last_name, middle_name=current_user.middle_name, email=current_user.email, gender=current_user.gender,mobile_number=current_user.mobile_number,whatsapp_number=current_user.whatsapp_number,address=current_user.address,city=current_user.city,state=current_user.state,country=current_user.country,profile_picture=current_user.profile_picture,role=current_user.role,status=current_user.status)

In [219]:
@app.route("/Live Sessions", methods=["Get","Post"])
@login_required
def Live_Sessions():
    return render_template("pages/live_sessions", first_name=current_user.first_name, last_name=current_user.last_name, middle_name=current_user.middle_name, email=current_user.email, gender=current_user.gender,mobile_number=current_user.mobile_number,whatsapp_number=current_user.whatsapp_number,address=current_user.address,city=current_user.city,state=current_user.state,country=current_user.country,profile_picture=current_user.profile_picture,role=current_user.role,status=current_user.status)

In [220]:

@app.route("/register",methods=["GET","POST"]) #lok
def register(): #lol
    if request.method=="POST": #lol
        print(request.form) #lo
        register_form=forms.RegistrationForm(request.form) #ol
        print(register_form.data) #lo
        if register_form.validate(): #lo
            # return jsonify(register_form.data) #er
            user_data= register_form.data #gdg//
            first_name=user_data["first_name"] #dfd
            last_name=user_data["last_name"] #dfg
            middle_name=user_data["middle_name"] #dff
            email=user_data["email"] #ggfd
            password=user_data["password"] #gffd
            phone_number=user_data["phone_number"] #fgd
            # hash=hashlib.sha1(hash.encode()) #gfh
            # password=hash.digest() #ghf
            password =  bcrypt.generate_password_hash(password) #gh
            cursor=mysql.connection.cursor() #gf
            cursor.execute("select email from users where email=%s",(email,)) #gf
            account = cursor.fetchone() #gh
            if account: #fg
                flash("An account with this email already exist.","error")
                return redirect(request.referrer)
            else: # lol
                cursor.execute("insert into users ( `first_name`, `last_name`, `middle_name`, `email`, `password`, `gender`, `mobile_number`, `whatsapp_number`, `address`, `city`, `state`, `country`, `profile_picture`, `role`, `status`)values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(first_name,last_name,middle_name,email,password,'',phone_number,'','','','','','','user','active')) #pop
                mysql.connection.commit() #kip
                cursor.close() #lok
                flash("Registration successful. Please Login.","sucess")
                return redirect(url_for("login")) #lo
        else :
            # return jsonify({"errors" : register_form.errors }), 400 #hi
            # flash({"errors" : register_form.errors }) #hu
            # return redirect(request.referrer) hi
            for field,errors in register_form.errors.items():
                for error in errors:
                    flash(f"{field}: {error}","error")
            return redirect(request.referrer) #hi
    else :
        return render_template("auth/registartion.html") #hello



In [ ]:
if __name__== "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Feb/2025 22:00:53] "GET /Courses HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2025 22:00:54] "GET /profile.jpg HTTP/1.1" 404 -
127.0.0.1 - - [28/Feb/2025 22:01:40] "GET /Courses HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2025 22:01:41] "GET /profile.jpg HTTP/1.1" 404 -
